In [170]:
assignments = []

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

boxes = cross(rows,cols)
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
 
diagonal1 = [[a[0]+a[1] for a in zip(rows,cols)]]
diagonal2 = [[a[0]+a[1] for a in zip(rows,cols[::-1])]]
diagonals = diagonal1 + diagonal2
#print(diagonals)

unitlist = row_units + column_units + square_units + diagonals
#print(unitlist)
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
#print(units)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find lists of boxes in the same *unit, with *len()=2 
    unit_boxes_2digits = [[box for box in unit if len(values[box]) == 2] for unit in unitlist]
    print(unit_boxes_2digits)
    #Using sample1 generates: [[], [], ['C9'], [], ['E8'], [], ['G8'], ['H8'], ['I8'], [], [], [], [], [], [], [], ['E8', 'G8', 'H8', 'I8'], ['C9'], [], [], ['C9'], [], [], ['E8'], [], [], ['G8', 'H8', 'I8'], ['H8'], []]
    
    #we should NOT eliminate boxes lists with #boxes with 2-digit values > 2
    unit_over2boxes_2digits = [i for i in unit_boxes_2digits if len(i)>1]
    print(unit_over2boxes_2digits)
    #generates = [['E8', 'G8', 'H8', 'I8'], ['G8', 'H8', 'I8']]
        
    #Now we need to iterate over each unit_boxes list to find equal values
    naked_twins = [[i for i in unit_boxes if values[i]==values[i+1]] for unit_boxes in unit_over2boxes_2digits]
    print(naked_twins) 
    #generates = TypeError: must be str, not int
    
    # Eliminate the naked twins as possibilities for their peers
    
    for box1,box2 in naked_twins:
        digits = values[box1]
        common_peers = list(peers[box1] & peers[box2])
        for peer in common_peers:
            if len(values[peer])>1 and peer!=box1 and peer!=box2:
                values[peer]= values[peer].replace(digits[0],'')
                values[peer]= values[peer].replace(digits[1],'')
    
    return values
  
    

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    grid = ['123456789' if i=='.' else i for i in  list(grid)]
    return dict(zip(boxes,grid))

def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    single_boxes = [i for i in values.keys() if len(values[i])==1]
    
    for box in single_boxes:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    
    return values

def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            d_place = [box for box in unit if digit in values[box]]
            if len(d_place)==1:
                values[d_place[0]]=digit
    return values

def reduce_puzzle(values):
    #reduce with eliminate and only_choice ********************NOT naked twins?
    
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        
        # Your code here: Use the Eliminate Strategy
        values=eliminate(values)
        # Your code here: Use the Only Choice Strategy
        values=only_choice(values)
        
        
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values=grid_values(grid)
    return search(values)
    
    


In [171]:
if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(solve(diag_sudoku_grid))

    
    before_twins={'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
    
    
    recommended_board={"G7": "1569", "G6": "134568", "G5": "13568", "G4": "134568", "G3":
"2", "G2": "34589", "G1": "7", "G9": "5689", "G8": "15", "C9": "56",
"C8": "3", "C3": "7", "C2": "1245689", "C1": "1245689", "C7": "2456",
"C6": "1245689", "C5": "12568", "C4": "1245689", "E5": "4", "E4":
"135689", "F1": "1234589", "F2": "12345789", "F3": "34589", "F4":
"123589", "F5": "12358", "F6": "123589", "F7": "14579", "F8": "6",
"F9": "3579", "B4": "1234567", "B5": "123567", "B6": "123456", "B7":
"8", "B1": "123456", "B2": "123456", "B3": "345", "B8": "9", "B9":
"567", "I9": "578", "I8": "27", "I1": "458", "I3": "6", "I2": "458",
"I5": "9", "I4": "124578", "I7": "3", "I6": "12458", "A1": "2345689",
"A3": "34589", "A2": "2345689", "E9": "2", "A4": "23456789", "A7":
"24567", "A6": "2345689", "A9": "1", "A8": "4", "E7": "159", "E6":
"7", "E1": "135689", "E3": "3589", "E2": "135689", "E8": "15", "A5":
"235678", "H8": "27", "H9": "4", "H2": "3589", "H3": "1", "H1":
"3589", "H6": "23568", "H7": "25679", "H4": "235678", "H5": "235678",
"D8": "8", "D9": "3579", "D6": "123569", "D7": "14579", "D4":
"123569", "D5": "12356", "D2": "12345679", "D3": "3459", "D1":
"1234569"}
    
    #x=naked_twins(before_twins1)
      
    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


In [172]:
display(recommended_board)

 2345689  2345689   34589  | 23456789  235678  2345689 |  24567      4        1    
  123456   123456    345   | 1234567   123567   123456 |    8        9       567   
 1245689  1245689     7    | 1245689   12568   1245689 |   2456      3        56   
---------------------------+---------------------------+---------------------------
 1234569  12345679   3459  |  123569   12356    123569 |  14579      8       3579  
  135689   135689    3589  |  135689     4        7    |   159       15       2    
 1234589  12345789  34589  |  123589   12358    123589 |  14579      6       3579  
---------------------------+---------------------------+---------------------------
    7      34589      2    |  134568   13568    134568 |   1569      15      5689  
   3589     3589      1    |  235678   235678   23568  |  25679      27       4    
   458      458       6    |  124578     9      12458  |    3        27      578   


In [173]:
display(naked_twins(recommended_board))

[[], [], ['C9'], [], ['E8'], [], ['G8'], ['H8'], ['I8'], [], [], [], [], [], [], [], ['E8', 'G8', 'H8', 'I8'], ['C9'], [], [], ['C9'], [], [], ['E8'], [], [], ['G8', 'H8', 'I8'], ['H8'], []]
[['E8', 'G8', 'H8', 'I8'], ['G8', 'H8', 'I8']]


TypeError: must be str, not int

In [141]:
recommended_board['H8'],recommended_board['I8']

('27', '27')